In [ ]:
%matplotlib inline
## coding=utf-8
import matplotlib.pyplot as plt
plt.rc('image', interpolation='nearest', origin='lower', cmap = 'hot')
rcParams = plt.rcParams.copy()
#from utils import colorbar,imshow

import numpy as np
import posixpath, glob, datetime, os, sys

from astropy import wcs as pywcs
from astropy.io import fits as pyfits

from scipy.spatial import cKDTree

import sep
#import cv2
from esutil import htm, coords

#import statsmodels.api as sm
from scipy.stats import binned_statistic_2d, binned_statistic
from skimage.measure import block_reduce
from skimage.util import view_as_blocks

from tqdm import tqdm,tqdm_notebook



In [ ]:
from pylab import *
import pyfits,glob,time
from subprocess import call
topdir='/Users/cslage/Research/LSST/code/GUI/'


thedir='/Users/cslage/Research/LSST/code/GUI/spot_sizes/'
filelist=sort(glob.glob(thedir+'bf_1.fits'))
print len(filelist)," files"


In [ ]:
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.x2=zeros([nstamps])
        self.y2=zeros([nstamps])
        self.xy=zeros([nstamps])
        self.imax=zeros([nstamps])

def BuildSpotList(fitsfilename, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 100
    xcoomax = 1900
    ycoomin = 100
    ycoomax = 1900
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename)
    img=pyfits.getdata(fitsfilename) 
    catname=fitsfilename[:-5]+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = img[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff
            spotlist.yoffset[n] = yoff

            x2sum = 0.0
            y2sum = 0.0
            xysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    x2sum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.x[i] - spotlist.xoffset[n]) * spotlist.data[i,j,n]
                    y2sum += (spotlist.y[j] - spotlist.yoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    xysum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.x2[n] = x2sum / datasum
            spotlist.y2[n] = y2sum / datasum
            spotlist.xy[n] = xysum / datasum
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.x2 = spotlist.x2[0:n]
    newspotlist.y2 = spotlist.y2[0:n]
    newspotlist.xy = spotlist.xy[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom




In [ ]:
spotlist = BuildSpotList(filelist[0], 100, 9, 9, 1.3, 1.7)

In [ ]:
print spotlist.nstamps
imshow(spotlist.data[:,:,33],interpolation='Nearest')

In [ ]:
n=22
from ngmix.observation import Observation, ObsList
obs = Observation(spotlist.data[:,:,n])
from ngmix.bootstrap import EMRunner


ngauss = 1    # number of gaussians to fit
Tguess = 4.0  # Ixx+Iyy in units defined by the wcs
ntry   = 5    # how many times to try with different random
              # guesses based on Tguess

em_pars={'maxiter':1000, 'tol':1.0e-6}

runner=EMRunner(obs, Tguess, ngauss, em_pars)

runner.go(ntry=ntry)

fitter=runner.get_fitter()
print type(fitter)
res=fitter.get_result()
gmix=fitter.get_gmix()
pars=gmix.get_full_pars()
print pars
sigma = gmix.get_sigma()
print "Sigma = %.4f"%sigma
print spotlist.xoffset[n], spotlist.yoffset[n], spotlist.x2[n], spotlist.y2[n], spotlist.xy[n]

In [ ]:
n=37
from ngmix.observation import Observation, ObsList
obs = Observation(spotlist.data[:,:,n])

pars=array([0.6,4.0,4.0,2.0,0.1,2.0])
gmix = ngmix.GMix(ngauss=1, pars=pars)
fitter=ngmix.em.GMixEM(obs)
fitter.go(gmix, 2.0, maxiter=1000)
res=fitter.get_result()
gmix=fitter.get_gmix()
pars=gmix.get_full_pars()
print pars
print spotlist.xoffset[n], spotlist.yoffset[n], spotlist.x2[n], spotlist.y2[n], spotlist.xy[n]
sigma = gmix.get_sigma()
print "Sigma = %.4f"%sigma


In [ ]:
n=37
from ngmix.observation import Observation, ObsList
obs = Observation(spotlist.data[:,:,n])

pars=array([0.6,4.0,4.0,2.0,0.1,2.0])
gmix = ngmix.GMix(ngauss=1, pars=pars)
print type(gmix)
mc = ngmix.fitting.MCMCSimple(obs, 'gauss', nwalkers=80, nsub=4)
print type(mc)
#model = ngmix.gmix.make_gmix_model(pars, 'GMIX_FULL')
#print type(model)
#model = ngmix.GMixModel(pars, 'gauss')
#model.ngauss = 1


In [ ]:
n=22
from ngmix.observation import Observation, ObsList
from ngmix.fitting import LMSimple
obs = Observation(spotlist.data[:,:,n])

fitter=LMSimple(obs,'gauss')
guess=array([4.0,4.0,1.0,1.0,4.0,2000])

fitter.go(guess)

res=fitter.get_result()

print res
print spotlist.xoffset[n], spotlist.yoffset[n], spotlist.x2[n], spotlist.y2[n], spotlist.xy[n]

In [ ]:
print type(pars)
import inspect
import ngmix
print inspect.getfile(ngmix)

In [ ]:
from ngmix.observation import Observation, ObsList
obs_list = []
for i in range(spotlist.nstamps):
    obj = Observation(spotlist.data[:,:,i])
    obs_list.append(obj)
ngmix_obs_list = ObsList(obs_list)
ngauss = 1    # number of gaussians to fit
Tguess = 4.0  # Ixx+Iyy in units defined by the wcs
ntry   = 5    # how many times to try with different random
              # guesses based on Tguess

em_pars={'maxiter':1000, 'tol':1.0e-6}

runner=EMRunner(ngmix_obs_list, Tguess, ngauss, em_pars)

runner.go(ntry=ntry)

fitter=runner.get_fitter()
res=fitter.get_result()
gmix=fitter.get_gmix()
pars=gmix.get_full_pars()
print pars
#print spotlist.xoffset[n], spotlist.yoffset[n], spotlist.x2[n], spotlist.y2[n], spotlist.xy[n]

In [ ]:
import ngmix
from ngmix.fitting import LMSimple, MCMCSimple

def srandu(num=None):
#  Generate random numbers in the symmetric distribution [-1,1]
    return 2*(np.random.random(num)-0.5)

def get_sigma(img, weight_map=None, xc=4.0, yc=4.0, nwalkers=80, burnin=400, nstep=400):
    guess=np.zeros((nwalkers, 6))

    if weight_map is None:
        weight_map = 1./np.sqrt(np.fabs(img))
        
    obs = ngmix.observation.Observation(img, weight=weight_map)
    mc = MCMCSimple(obs, 'gauss', nwalkers=nwalkers, nsub=4)
    guess[:,0] = xc + 0.1*srandu(nwalkers)
    guess[:,1] = yc + 0.1*srandu(nwalkers)
    guess[:,2] = 0. + 0.1*srandu(nwalkers)
    guess[:,3] = 0. + 0.1*srandu(nwalkers)
    guess[:,4] = 0.32 * (1.0 + 0.1*srandu(nwalkers)) # 0.32 = 2*(0.4)**2
    guess[:,5] = np.sum(img) * (1.0 + 0.1*srandu(nwalkers))

    pos = mc.run_mcmc(guess, burnin)
    pos = mc.run_mcmc(pos,   nstep)

    mc.calc_result()

    result = mc.get_result()

    if result['flags'] == 0:
        return result['pars'][0], result['pars'][1], np.sqrt(result['pars'][4]/2.0)
    else:
        return 0.0, 0.0, 0.0

In [ ]:
x1s,y1s = [],[]
sigmas = []

for i in range(10):
        img = spotlist.data[:,:,i]

        xc,yc = 3.5, 3.5

        # Get peak position using MCMC
        yc,xc,sigma = get_sigma(img, xc=yc, yc=xc, nstep=4000)
        
        # If the fit properly converged, store this spot parameters
        if xc > 0 and xc < 6 and yc > 0 and yc < 6 and sigma > 0 and sigma < 10:
            sigmas.append(sigma)
            x1s.append(xc)
            y1s.append(yc)
            
        
sigmas,x1s,y1s = [np.array(_) for _ in sigmas,x1s,y1s]

In [ ]:
print sigmas
print sigmas.mean()
hist(sigmas, bins = 10)